Use the CNN and MLP models

使用CNN和MLP模型

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"

path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"
path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"

path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"


mark_num = "25"
meta_name = "meta_data.csv"
list_color = ["#ee877c", "#8bd0e3", "#6abeae", "#808eaf", "#f7bba8", "#acb4cc", "#b5e0d5", "#e86462", "#a89687"]

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, Subset
import torch.nn.functional as F
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import itertools
# CNN
def start_cnn(df_o):
    data = df_o.copy()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X = data.drop('value', axis=1)
    y = data['value']
    X = X.values.reshape(X.shape[0], X.shape[1], 1)
    y = y.values

    list_batch_size = [1024,2048,4096]
    list_conv1_outcome = [32, 64]
    list_fc1_outcome = [128,258]
    list_learn_rate = [0.001]
    list_epoch_range = [10000]
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    start_var_num = data.shape[1]-1


    def train_model(params, train_ids, test_ids):
        batch_size, fc1_outcome, fc2_outcome, learn_rate, epoch_range = params
        print("all param:", batch_size, fc1_outcome, fc2_outcome, learn_rate, epoch_range)
        class Net(nn.Module):
            def __init__(self):
                super(Net, self).__init__()
                self.conv1 = nn.Conv1d(start_var_num, fc1_outcome, 3, padding=1)  # Input channels, output channels, kernel size
                self.pool = nn.MaxPool1d(1)  # Kernel size
                self.fc1 = nn.Linear(fc1_outcome, fc2_outcome)  # Input features, output features
                self.dropout = nn.Dropout(0.0001)
                self.fc2 = nn.Linear(fc2_outcome, 1)

            def forward(self, x):
                x = self.pool(F.relu(self.conv1(x)))
                x = x.view(-1, fc1_outcome)
                x = F.relu(self.fc1(x))
                x = self.dropout(x)
                x = self.fc2(x)
                return x.view(-1)

        model = Net().to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learn_rate)

        train_subset = Subset(TensorDataset(X, y), train_ids)
        test_subset = Subset(TensorDataset(X, y), test_ids)

        train_loader = torch.utils.data.DataLoader(
                        train_subset, 
                        batch_size=batch_size)
        test_loader = torch.utils.data.DataLoader(
                        test_subset,
                        batch_size=batch_size)

        min_loss = float('inf') 
        epochs_no_improve = 0  
        for epoch in range(epoch_range): 
            total_loss = 0  
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                total_loss += loss.item()  

            avg_loss = total_loss / len(train_loader) 

            if avg_loss < min_loss:
                min_loss = avg_loss 
                epochs_no_improve = 0 
            else:
                epochs_no_improve += 1 

            if epochs_no_improve == 100:
                print('Early stopping after {} epochs'.format(epoch+1))
                break  

        model.eval()
        predictions = []
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.detach().cpu().numpy())

        r2 = r2_score(y[test_ids].numpy(), predictions)
        mse = mean_squared_error(y[test_ids].numpy(), predictions)
        #print('R^2 score:', r2)
        #print('MSE score:', mse)
        return r2, mse

    def grid_search(params):
        print('------------------CNN stsrt-----------------')
        kfold = KFold(n_splits=10, shuffle=True, random_state=202406)
        param_combinations = list(itertools.product(*params.values()))
        results = []
        for params in param_combinations:
            r2_scores = []
            mse_score = []
            for fold, (train_ids, test_ids) in enumerate(kfold.split(X)):
                print('fold', fold)
                r2, mse = train_model(params, train_ids, test_ids)
                r2_scores.append(r2)
                mse_score.append(mse)
            avg_r2 = np.mean(r2_scores)
            # std_r2 = np.std(r2_scores)
            avg_mse = np.mean(mse_score)
            print('Average R^2 score:', avg_r2)
            print('Average MSE score:', avg_mse)
            results.append((params, r2_scores, avg_r2, avg_mse))
        return results

    param_grid = {
        'batch_size': list_batch_size,
        'conv1_outcome': list_conv1_outcome,
        'fc1_outcome': list_fc1_outcome,
        'learn_rate': list_learn_rate,
        'epoch_range': list_epoch_range
    }

    results = grid_search(param_grid)

    results_df = pd.DataFrame(results, columns=['params', 'r2_scores', 'avg_r2', 'avg_mse'])
    results_df[['batch_size', 'conv1_outcome', 'fc1_outcome', 'learn_rate', 'epoch_range']] = pd.DataFrame(results_df['params'].tolist(), index=results_df.index)
    results_df = results_df.drop('params', axis=1)
    results_df['rank'] = results_df['avg_r2'].rank(ascending=False)
    print('------------------CNN END-----------------')
    return results_df


def start_mlp(df_o):
    data = df_o.copy()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X = data.drop('value', axis=1)
    y = data['value']
    X = data.drop('value', axis=1).values
    y = y.values
    start_var_num = data.shape[1]-1
    list_batch_size = [1024,2048,4096]
    list_conv1_outcome = [32, 64]
    list_fc1_outcome = [128,258]
    list_learn_rate = [0.001]
    list_epoch_range = [10000]

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    def train_model(params, train_ids, test_ids):
        batch_size, fc1_outcome, fc2_outcome, learn_rate, epoch_range = params
        print("all param:", batch_size, fc1_outcome, fc2_outcome, learn_rate, epoch_range)

        class MLP(nn.Module):
            def __init__(self):
                super(MLP, self).__init__()
                self.fc1 = nn.Linear(start_var_num, fc1_outcome)  # Input features, output features
                self.fc2 = nn.Linear(fc1_outcome, fc2_outcome)  # Input features, output features
                self.fc3 = nn.Linear(fc2_outcome, 512)
                self.fc4 = nn.Linear(512, 1)

            def forward(self, x):
                x = F.relu(self.fc1(x))
                x = F.relu(self.fc2(x))
                x = F.relu(self.fc3(x))
                x = self.fc4(x)
                return x.view(-1)

        model = MLP().to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learn_rate)

        train_subset = Subset(TensorDataset(X, y), train_ids)
        test_subset = Subset(TensorDataset(X, y), test_ids)

        train_loader = torch.utils.data.DataLoader(
                        train_subset, 
                        batch_size=batch_size)
        test_loader = torch.utils.data.DataLoader(
                        test_subset,
                        batch_size=batch_size)

        min_loss = float('inf') 
        epochs_no_improve = 0  
        for epoch in range(epoch_range): 
            total_loss = 0  
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                total_loss += loss.item()  

            avg_loss = total_loss / len(train_loader) 

            if avg_loss < min_loss:
                min_loss = avg_loss 
                epochs_no_improve = 0 
            else:
                epochs_no_improve += 1 

            if epochs_no_improve == 100:
                print('Early stopping after {} epochs'.format(epoch+1))
                break  

        model.eval()
        predictions = []
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.detach().cpu().numpy())

        r2 = r2_score(y[test_ids].numpy(), predictions)
        mse = mean_squared_error(y[test_ids].numpy(), predictions)
        # print('R^2 score:', r2)
        return r2, mse

    def grid_search(params):
        print('------------------MLP stsrt-----------------')
        kfold = KFold(n_splits=10, shuffle=True, random_state=202406)
        param_combinations = list(itertools.product(*params.values()))
        results = []
        for params in param_combinations:
            r2_scores = []
            mse_score = []
            for fold, (train_ids, test_ids) in enumerate(kfold.split(X)):
                print('fold', fold)
                r2, mse = train_model(params, train_ids, test_ids)
                r2_scores.append(r2)
                mse_score.append(mse)
            avg_r2 = np.mean(r2_scores)
            # std_r2 = np.std(r2_scores)
            avg_mse = np.mean(mse_score)
            print('Average R^2 score:', avg_r2)
            print('Average MSE score:', avg_mse)
            results.append((params, r2_scores, avg_r2, avg_mse))
        return results

    param_grid = {
        'batch_size': list_batch_size,
        'conv1_outcome': list_conv1_outcome,
        'fc1_outcome': list_fc1_outcome,
        'learn_rate': list_learn_rate,
        'epoch_range': list_epoch_range
    }

    results = grid_search(param_grid)

    results_df = pd.DataFrame(results, columns=['params', 'r2_scores', 'avg_r2', 'avg_mse'])
    results_df[['batch_size', 'conv1_outcome', 'fc1_outcome', 'learn_rate', 'epoch_range']] = pd.DataFrame(results_df['params'].tolist(), index=results_df.index)
    results_df = results_df.drop('params', axis=1)
    results_df['rank'] = results_df['avg_r2'].rank(ascending=False)
    print('------------------MLP END-----------------')
    return results_df

def get_nn_best_model(df_o):
    df = df_o.copy()
    grouped = df.groupby(['model'])
    results = pd.DataFrame()
    for name, group in grouped:
        max_row = group[group['avg_r2'] == group['avg_r2'].max()]
        results = pd.concat([results, max_row])
    return results

### start

In [3]:
path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'
data = pd.read_csv(path_semdata + "sem_sw_s7_au_to_"+mark_num+"_avg.csv")
data['year'] = (data['year'] - 2000) / (2020 - 2000)
columns_to_drop = ['lon_grid', 'lat_grid']
data = data.drop(columns=columns_to_drop)
print(data.columns)

Index(['lict', 'skt', 'cth', 'i10fg', 'u100', 'SF6', 'dl', 'ctp', 'licd',
       'TNR_Ship_CO2_E', 'fluorite_consumption', 'sro', 'nships_smoothed',
       'si10', 'TOTALS_CO2_E', 'z', 'v10', 'port_distance', 'ref_liq',
       'wrap_consumption', 'CF4', 'HFC', 'stl1', 'e', 'paper_consumption',
       'human_footprint', 'C2F6', 'night_lights', 'tp', 'sp', 'u10', 'cl',
       'global_salinity', 'ctt', 'tcsw', 'cfc', 'v100', 'HCFC',
       'estuary_distance', 'GDP', 'str', 'tcrw', 'sshf', 'mine_distance',
       'ssro', 'swvl1', 'ltlt', 'ref_ice', 'SWD_INC_CO2_E', 'population',
       'ssr', 'year', 'log_Koil_air', 'density', 'po_f_carbon',
       'melting_point', 'boiling_point', 'solubility', 'po_xlogp', 'po_carbon',
       'log_Kow', 'log_KHxd_air', 'log_Kaw', 'log_Koa_wet', 'po_m_w',
       'log_Koil_w', 'log_Koc', 'log_Koa_dry', 'value', 'po_chain'],
      dtype='object')


In [4]:
df_cnn_result = start_cnn(data)
df_cnn_result['model'] = 'CNN'
df_cnn_result.to_csv(path_sw_rfecv_data + 'cnn_cv_result.csv',index=False)


df_mlp_result = start_mlp(data)
df_mlp_result['model'] = 'MLP'
df_mlp_result.to_csv(path_sw_rfecv_data + 'mlp_cv_result.csv',index=False)

df_nn_result = pd.concat([df_cnn_result, df_mlp_result],axis=0)



results = get_nn_best_model(df_nn_result)
results.to_csv(path_sw_rfecv_data + 'nn_cv_best.csv', index=False)

------------------CNN stsrt-----------------
fold 0
all param: 1024 32 128 0.001 10000
Early stopping after 2201 epochs
fold 1
all param: 1024 32 128 0.001 10000
Early stopping after 2180 epochs
fold 2
all param: 1024 32 128 0.001 10000
Early stopping after 1746 epochs
fold 3
all param: 1024 32 128 0.001 10000
Early stopping after 2405 epochs
fold 4
all param: 1024 32 128 0.001 10000
Early stopping after 1899 epochs
fold 5
all param: 1024 32 128 0.001 10000
Early stopping after 2078 epochs
fold 6
all param: 1024 32 128 0.001 10000
Early stopping after 2908 epochs
fold 7
all param: 1024 32 128 0.001 10000
Early stopping after 2429 epochs
fold 8
all param: 1024 32 128 0.001 10000
Early stopping after 2198 epochs
fold 9
all param: 1024 32 128 0.001 10000
Early stopping after 1141 epochs
Average R^2 score: 0.6894096517144593
Average MSE score: 0.31035626
fold 0
all param: 1024 32 258 0.001 10000
Early stopping after 2193 epochs
fold 1
all param: 1024 32 258 0.001 10000
Early stopping after

In [5]:
path_lr_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/lr_forecast/'
data2 = pd.read_csv(path_semdata + "sem_lrsws7_au_to_25_avg_full.csv")
data2['year'] = (data2['year'] - 2000) / (2020 - 2000)
columns_to_drop = ['lon_grid', 'lat_grid']
data2 = data2.drop(columns=columns_to_drop)
print(data2.shape)

(18497, 77)


In [6]:
df_cnn_result2 = start_cnn(data2)
df_cnn_result2['model'] = 'CNN'
df_cnn_result2.to_csv(path_lr_rfecv_data + 'cnn_cv_result.csv',index=False)


df_mlp_result2 = start_mlp(data)
df_mlp_result2['model'] = 'MLP'
df_mlp_result2.to_csv(path_lr_rfecv_data + 'mlp_cv_result.csv',index=False)

df_nn_result2 = pd.concat([df_cnn_result2, df_mlp_result2],axis=0)
df_nn_result2.to_csv(path_lr_rfecv_data + 'nn_cv_result.csv',index=False)

results2 = get_nn_best_model(df_nn_result)
results2.to_csv(path_lr_rfecv_data + 'nn_cv_best.csv', index=False)

------------------CNN stsrt-----------------
fold 0
all param: 1024 32 128 0.001 10000
Early stopping after 684 epochs
fold 1
all param: 1024 32 128 0.001 10000
Early stopping after 2099 epochs
fold 2
all param: 1024 32 128 0.001 10000
Early stopping after 539 epochs
fold 3
all param: 1024 32 128 0.001 10000
Early stopping after 2234 epochs
fold 4
all param: 1024 32 128 0.001 10000
Early stopping after 742 epochs
fold 5
all param: 1024 32 128 0.001 10000
Early stopping after 2981 epochs
fold 6
all param: 1024 32 128 0.001 10000
Early stopping after 561 epochs
fold 7
all param: 1024 32 128 0.001 10000
Early stopping after 424 epochs
fold 8
all param: 1024 32 128 0.001 10000
Early stopping after 852 epochs
fold 9
all param: 1024 32 128 0.001 10000
Early stopping after 1854 epochs
Average R^2 score: 0.771211672053121
Average MSE score: 0.22801217
fold 0
all param: 1024 32 258 0.001 10000
Early stopping after 1927 epochs
fold 1
all param: 1024 32 258 0.001 10000
Early stopping after 2847 e